Laden der Daten von Kaggle und die eigene `no_flood_images`

In [ ]:
from pathlib import Path
from PIL import Image, UnidentifiedImageError
import numpy as np
import os

no_flood_image_dir = Path("no_flood_images/Image")
no_flood_mask_out_dir = no_flood_image_dir / "masks"
no_flood_label_out_dir = no_flood_image_dir / "labels"

no_flood_mask_out_dir.mkdir(parents=True, exist_ok=True)
no_flood_label_out_dir.mkdir(parents=True, exist_ok=True)

IMG_HEIGHT, IMG_WIDTH = 224, 224

def load_no_flood_data():
    images, masks = [], []
    for img_file in sorted(no_flood_image_dir.glob("*.jpg")):
        try:
            img = Image.open(img_file).resize((IMG_WIDTH, IMG_HEIGHT)).convert("RGB")
        except UnidentifiedImageError:
            print(f"Skipped invalid image: {img_file.name}")
            continue
        images.append(np.array(img))
        masks.append(np.zeros((IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8))  # empty mask
    return images, masks

no_flood_images, no_flood_masks = load_no_flood_data()


### Data augmentation

Die Daten wurde erweitert durch folgende techniken:
- flipping
- cropping
- rotation
- brightness adjustment


Ich habe dafür die [Albumentations](https://albumentations.ai/docs/) Bibliothek verwendet.

Ich hab mich gegen "coarse dropout" entschieden, weil es unrealistsch ist, denke ich. Man bekommt nicht ein Foto, an dem ein Teil fehlt. Aber vielleicht, ist es Sinnvoll.

#### Für die Unterscheidung von Himmel und Wasser zusätzlich noch ergänzt:

Also um mit überbelichteten und kontrastarmen Bildern besser umzugehen:
- `gamma_aug`: Gamma-Korrektur RandomGamma(gamma_limit=(80, 120))

RandomGamma verstärkt und verändert Kontrast und Helligkeit auf nicht-lineare Weise.


In [ ]:
import albumentations as A
import os
from pathlib import Path
import cv2
import numpy as np


def augment(images, masks):
    augmented_images = []
    augmented_masks = []

    for img, msk in zip(images, masks):
        augmented_images.append(img)
        augmented_masks.append(msk)

        transforms = [
            A.Resize(height=224, width=224, p=1.0),
            A.GridDistortion(p=1.0, distort_limit=0.6),
            A.OpticalDistortion(p=1.0, distort_limit=1, mode='camera'),
            A.ElasticTransform(p=1.0, alpha=120, sigma=120),
            A.HorizontalFlip(p=1.0),
            A.RandomRotate90(p=1.0),
            A.RandomBrightnessContrast(p=1.0),
            A.GaussianBlur(p=1.0, sigma_limit=(1, 1.5)),
            A.RandomGamma(gamma_limit=(80, 120), p=1.0)
        ]

        for transform in transforms:
            result = transform(image=img, mask=msk)
            augmented_images.append(result['image'])
            augmented_masks.append(result['mask'])

    return augmented_images, augmented_masks

aug_noflood_images, aug_noflood_masks = augment(no_flood_images, no_flood_masks)

existing_files = sorted(list(no_flood_image_dir.glob("*.jpg")))
start_idx = len(existing_files)

# save image & mask
for i, (img, mask) in enumerate(zip(aug_noflood_images, aug_noflood_masks), start=start_idx):
    new_img_name = f"no_0{i:04d}.jpg"
    new_mask_name = f"no_0{i:04d}.png"

    cv2.imwrite(str(no_flood_image_dir / new_img_name), img)
    cv2.imwrite(str(no_flood_mask_out_dir / new_mask_name), mask)

#### Datenvorbereitung

Create lables from masks -> polygons (contours) in txt


In [ ]:
# Generate YOLO polygon labels
mask_files = sorted(no_flood_mask_out_dir.glob("*.png"))  # augmented + originals masks
for mask_path in mask_files:
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for (x, y) in cnt.squeeze():
                polygon += [x / W, y / H]
            polygons.append(polygon)

    label_path = no_flood_label_out_dir / f"{mask_path.stem}.txt"
    with label_path.open("w") as f:
        if polygons:
            for poly in polygons:
                f.write("0 " + " ".join(map(str, poly)) + "\n")
        else:
            pass # write empty file and yolo can interpret it

print(f"Wrote {len(list(no_flood_label_out_dir.iterdir()))} label files to {no_flood_label_out_dir}")

In [ ]:
# show how this created txt labels look like

import random

all_label_files = list(no_flood_label_out_dir.glob("*.txt"))
random_label = random.choice(all_label_files)
print(f"\n--- content of a random label file {random_label.name} ---")
with random_label.open() as f:
    print(f.read())